# Docker

<p align=center><a href=https://www.docker.com><img src=images/Docker_Logo.png width=400></a></p>

## Introduction


You have probably encountered an issue where the main problem is the compatibiity between the application you are running and your Operating System, or the installed packages. Wouldn't it be nice if all applications have a common ground that can be executed in any operating sytem? 

The solution is [Docker](https://www.docker.com), a tool that <i>container</i>ises your application so that it can ran in any environment. Containerising is the process of using containers that hold your application and all your dependencies in a single environment.

This idea was an extension of LXC (Linux Containers) and their usage, and __revolutionized the way we deploy software__.

__What can I use Docker for?__
- Standardized development environment across many workstations
- Consistent deployment of applications
- Full reproducibility (e.g. for research)

### Intro to Containers and Images

But wait... What is a container? You can think of a container as a Virtual Machine (VM), however, whilst a VM virtualises the hardware (how much RAM, memory you have), a container only virtualises the Operating System (OS). However, Docker doesn't just make a copy of the OS you want to work with, it will provide the necessary tools that will make the specific OS run. 

So, let's say for example that your application runs in the latest ubuntu version. Docker will NOT install the latest version each time you need to run your application, it will simply get the tools necessary to run that version without installing a whole OS. 

But, you might be wondering that even the tools might need a time to be installed right? That's correct, but the good thing about Docker is that it install them only once (unless you unistall it), and next time you want to run the same application, Docker will know where these tools have been installed. To do so, Docker relies on a powerful tool named Docker images, which are the templates to run containers. 

You can think of Docker images as Python classes, and containers as the Python instances. The image is the blueprint that indicates all the steps needed to run the container in which your application is held. The first time you `build` the image, those tools you need to run your application will be downloaded and installed, and Docker will know how to access those tools when you run you application again. 

One nifty trick Docker does to be more efficient is that those tools can be shared between images, so if you create another image where those tools are used again, Docker won't download or install them, because it knows how to find them.

> <font size=+1>Docker allows us to package our code, apps etc. with all the necessary dependencies in a self-contained environment called a `Docker image`. We can then create instances of these images, which we call `Docker containers` </font>

### Why are containers so important?

Now that you know what a container is, we can have a summary of why they are so important (and powerful)

Let's see how containers changed the deployment landscape:

<p align=center><img src=images/container_evolution.svg width=800></p>

When deploying a regular application, the application is quite lightweight because we just deploy the code for running the applcation, but we don't (usually) have in mind the operating system that application. If we wanted, say for example, deploy the whole virtual machine, we would need to install and launch the whole operating system everytime we want to run the application. Containers takes the best of both worlds and combines the lightweightness of deploying an application the traditional way with the fact that we can take into account that it can run in multiple applications.

__Benefits__:
- Containers are more lightweight than VMs
- They are __immutable__, meaning that their content won't change once we create the image
- You can easily create or destroy a container whenever you need it.
- All your application's necessities can be packed within the container
- Containers are reproducible since they run the same everywhere, regardless of the host OS.
- __Micro-services__ - app can be broken into multiple separate containers communicating with each other:
    - We only change image we need
    - No need to deploy everything a-new
    - Easily switchable components 

## Setting up Docker


Important! Before starting the rest of the notebook, it is recommended to use VSCode in your local machine, since the files you will run need the Docker engine to containerise your applications locally.

When creating a Docker image, your computer will need to create containers that, even though they are not VMs, they still need to allocate a memory slot (it's more complicated than `memory slots`, but for simplicity reasons, let's leave it like this) that will run certain tools that your OS might not support. 

In order to grant access to those memory slots, your computer needs to start an engine that creates the containers within your computer. Each OS has different ways to do so, for example, for Windows and iOS, you need to install Docker Desktop that will take care of creating the corresponding engine that will create the containers. On the other hand, Linux distributions are much more flexible, and you can run Docker just by installing the engine to create the containers.

Here we are going to see how to install Docker for each OS. 

<details>
    <summary> <font size=+2> For Linux Users </font> </summary>

To use Docker on Linux, you need to install Docker engine, which is the core technology of Docker. To insall it, go to the following webpage and follow the [instructions](https://docs.docker.com/engine/install/centos/). There, you will find how to install Docker based on your distribution.

If you are using Ubuntu, you can simply go to this [website](https://docs.docker.com/engine/install/ubuntu/). Make sure you follow the instructions that installs Docker engine `Using the repository`

<p align=center> <img src=images/Docker_Engine.png width=400> </p>

</details>

<details>
    <summary> <font size=+2>  For Mac Users and any Windows Users that doesn't have Home Edition </font> </summary>

In order to use Docker on Mac, you need to install Docker Desktop. For both Mac and Windows, you can download it from this [website](https://docs.docker.com/desktop/)

<p align=center> <img src=images/Docker_install.png width=400> </p>

After that, select the operating system you are using. Remember that if you are using Windows Home Edition, you need to refer to the next section (in this notebook) to complete the installation.

</details>

<details>
    <summary> <font size=+2>  For Windows Home Edition Users </font> </summary>

Unfortunatelly, Windows Home edition is the worst for installing anything that requires accessing your kernel. Luckily, there is (complicated) workaround this. You need to install the Hyper-V-Enabler, which grants permission to Docker for accessing your kernel. To download it, click this [link](https://aicore-files.s3.amazonaws.com/Foundations/Hyper-V-Enabler.bat)

Then, in the Windows searchbar, look for 'Turn Windows Features On or Off', and enable the following:
- Hyper-V Management Tools
- Hyper-V Platform
- Windows Hypervisor Platfor
- Windows Subsystem for Linux

<p align=center> <img src=images/Docker_Home_Edition.png width=400> </p>

You also might need to install the latest version of WSL. So you can install it using the following [file](https://aicore-files.s3.amazonaws.com/Foundations/wsl_update_x64.msi)

After that, follow the instructions included in the `For Mac Users and any Windows Users that doesn't have Home Edition`
</details>

For all versions, once you installed it, make sure the installation went fine by running `docker --version` in your CLI

### Docker Hub

So, we have been talkin about containers and how they set up a common ground for all applications. Many other users have the images to create those containers available for the rest of the world, so everytime you need to run an application, you don't have to do it from scratch. 

> <font size=+1> Docker Hub is a repository that stores Docker images from users all across the world </font>

You can think of Docker Hub as the GitHub for repositories, or the Pypi for Python packages. 

We will get more in depth with Docker Hub, but for now, let's create an account that we will eventually need to upload our images.

First, go to the [Docker Hub website](https://hub.docker.com) and create an account

<p align=center> <img src=images/Docker_Hub.png width=600> </p>

We will go back to DockerHub later in this notebook, and you will need to sign in through your terminal, so make sure you remember the password you used!

For now, one thing to bear in mind: you are going to use base images to create your own Docker images, and those base images are stored in Docker Hub.

## Docker Images and Dockerfile


> <font size=+1>Docker image are the instructions needed to create an instance of the Docker container</font>

Thus, Docker images are essentially a set of steps that the Docker engine will take to create the environment we need to run our application. Those steps are declared in a file named `Dockerfile`, which is a special type of file that Docker will look for whenever you want to build an image. 

`Dockerfile` doesn't contain any extension, the name of the file is literally `Dockerfile`, but you might use it as the extension. For example, if the Dockerfile specifies the steps to create an image for an API image, you can call it `api.Dockerfile`

In VSCode, when you create a Dockerfile, it will automatically recognize it as a Dockerfile, and you will notice thanks to the characteristic whale icon.

<p align=center> <img src=images/Docker_icon.png width=200> </p>

Once you create the Dockerfile, you can start containerising your application, but of course, you need to specify the commands you want Docker to run. 

Thus, let's take a look at what you can do inside a Dockerfile by looking at an example. 

Dockerfiles will contain instructions, such as `FROM`, `RUN`, `CMD`, `COPY`... The capitalised words starting each line in the Dockerfile are called __instructions__ and are basically commands, followed by arguments (like in terminal), which the docker build command knows how to execute. Docker build runs each of these instructions in turn to create the docker image.

### Your first Docker image

In this example you will create a Docker image that runs the celebrity_births scraper you can find in the `Software Design and Testing` module. In case you haven't completed that part yet (or you forgot where you put the file), you can download the files [here](https://aicore-files.s3.amazonaws.com/Foundations/DevOps/celebrity_example.zip)

After you download the file, `cd` to that folder and create the Dockerfile. Call the file `Dockerfile` and let's dive into it.

Inside the Dockerfile start writing this:

```Dockerfile
FROM python:3.8-slim-buster
```

Usually, Docker images are built from a pre-built image Docker can find on Docker Hub. The pre-built image usually contains some dependencies. For example, a common one is to use an image with Python installed. You can download and run the pre-built image using the `FROM` clause as we see above. 

So, with the first command we added, we will start creating our image with the necessary Python dependencies.

The next thing we need is to install or copy what we want inside the docker container. Remember that the directory you add is relative to the position Dockerfile is.

In your Dockerfile add the following line

```
COPY . . 
```

This will copy everything in the Dockerfile directory (`requirements.txt` and the `scraper` folder) inside the container.

It's very important to understand this step. When you build the image, you are going to copy your files inside the container, and it will be like they will be in another computer. Think about the container as a separate computer where you will copy the files. So at this point, it will be like having another mini computer with Python installed and your scraper in it

One more thing you need before running the scraper is installing your python packages, like beautifulsoup and requests. Luckily, we also copied the requirements file into the image, so we can run it directly.

```
RUN pip install -r requirements.txt
```

We are almost there! The only thing left to do is running the python script. We can't use the `RUN` clause here, because `RUN` is executed when the image is built. We need a command that is executed when we run the image, and that clause is `CMD`

```
CMD ["python", "scraper/celebrity_scraper.py"]
```

This clause has many ways to be declared, in this case, we are using square brackets, and the first item is the executable (`python`) and the rest of items are the parameters (files)

Now we are ready to build the image! In your command line interface, if you are not in the `celebrity_example` directory, move into it. Then, we need to use the `build` command from Docker. It has the following syntax:

`docker build [OPTIONS] [Dockerfile path]` 

You can check the options [here](https://docs.docker.com/engine/reference/commandline/build/). One of the common options you may want to use is the -t flag, to give a `tag` to our image. That way, the image will have a name. 

Since we are in the same directory as the Dockerfile, the Dockerfile path is simply a dot (`.`)

Run the following command in your command line

In [ ]:
docker build -t celebrities:latest .

The `latest` after the name of our image is the tag (or the version if you prefer)

We have just created our first image! Let's check if the image has been properly created by running the following cell

In [ ]:
docker images # show our current images on this machine

<p align=center> <img src=images/Docker_images.png width=600> </p>

Ok, so we have just built an image, let's run it. We can run an image using the following syntax:

`docker run [OPTIONS] IMAGE[:TAG|@DIGEST] [COMMAND] [ARG...]`

Let's try running our celebrities image running the following command:

`docker run celebrities`

This will throw an error, because the script expects an input, but right now, the image is running in a non-interactive mode. To change that, we need to add the options -i and -t. `-i` will keep the STDIN open, and `-t` will make the process interactive

<p align=center> <img src=images/Docker_run_error.png width=600> </p>

`docker run -it celebrities`

<p align=center> <img src=images/Docker_run.png width=600> </p>

Success! you can use this image everywhere now, regardless of the OS and dependencies installed in it. But how do you distribute it? Using Docker Hub!

You have already created an account on Docker Hub, so now you can go to your command line and run:

```
docker login
```

Then enter your credentials

The images you push to docker hub needs to have a specific name: 
```
<username>/<image_name>:<tag>
```
So, let's create a copy of the existing image with a new name. We can use the docker tag command to do so. The syntax is as follow:

```
docker tag <Image_Id> <New name>
```

Image_id can be seen when you run `docker image` (See picture above), let's run the following command. In my case, my username is ivanyingx, so change that with your username, and the Image_id is 82a51cbd4876:

```
docker tag 82a51cbd4876 ivanyingx/celebrities:v1
```

Then, you can check that the image has been properly created by running docker images again

<p align=center> <img src=images/Docker_tag.png width=600> </p>

By the way, you can also check this information in the Docker Desktop if you are on Mac or Windows:

<p align=center> <img src=images/Docker_Desktop.png width=600> </p>


Finally, it's time to push the image to Docker Hub. Pushing an image is very similar to pushing a repository to github, simply use docker push!

`docker push ivanyingx/celebrities:v1`

You can check that your image has been uploaded by going to your docker hub account:

<p align=center> <img src=images/Docker_Hub_example.png width=600> </p>


If any user wants to run your container, they can do it directly running it on their local machines. For example, in this case, you can run _my_ image by running:

`docker pull ivanyingx/celebrities` This will download the image

and then 

`docker run ivanyingx/celebrities` This will run the image

You can also run directly `docker run ivanyingx/celebrities`, which will perform both operations.

Congratulations! You have created and pushed your first Docker image! The rest of your Docker path will consist on practicing (a lot!). The rest of the notebook dives deeper in the concepts we have seen so far, adding some commands you might find useful in the Dockerfile

# Second Part

### Context & .dockerignore


"The build context is the set of files located at the specified PATH or URL. Those files are sent to the Docker daemon during the build so it can use them in the filesystem of the image." By default, the docker context is the location from which `docker build` is executed.

> When we build the image in a given directory, __everything is added recursively to the context__ (so it can be copied into image, like above)

`Dockerfile` might be surrounded by a lot of files and it takes time to copy them into `Docker` system (might even crash if there are too many files!)

Because of that `.dockerignore` file can be specified (really similar to `.gitignore`):

In [ ]:
__main__.py
requirements.txt

## Docker Containers


> <font size=+1>Containers are instantiations of images</font>

As we have built our `repository/python_image` we can create and run container from it:

In [ ]:
docker run repository/python_image:latest --help

### Thinking of containers


> One should think of the containers as __standalone units__ (like applications) __having single responsibility__

Examples could be (but are not limited to):
- MySQL database container (other containers connect to it via `EXPOSE`d ports)
- Data preprocessing creating a single artifact (preprocessed dataset)
- Neural network training creating a single artifact (neural network)

__Never try to fit everything into a single container!__

In the above case, our container is similar to simply running `python` from the command line

> Containers should be immutable (their internal state is always the same)

This allows us to:
- Destroy and recreate containers quickly
- Always be in a well-defined state

## Dockerfile commands (instructions)


> Docker provides a couple commands, which allows us to work with in a similar fashion to command line

__Each command creates a new layer__:
- Images can be built from any layer upwards
- __Layers are cached and reused by consecutive builds__
- __Layers can be reused between different images__

### [FROM](https://docs.docker.com/engine/reference/builder/#from)


> `FROM [--platform=<platform>] <image>[:<tag>] [AS <name>]`

- Starts __build stage__ of an image
- Specifies base image (like `Ubuntu`, `node`, `conda`) which defines what one can do in the image
- `AS` defines name for the image, we will see the usage during [multi-stage builds](https://docs.docker.com/develop/develop-images/multistage-build/).

It can be mixed with `ARG` (which allows us to pass this value from a command line) like this:

In [ ]:
# Version is out of build stage
ARG VERSION=latest
# Here build stage starts
FROM busybox:$VERSION

# Gets version into build stage
ARG VERSION
RUN echo $VERSION > image_version

### [RUN](https://docs.docker.com/engine/reference/builder/#run)


> Runs specified command __during build stage__ (e.g. installing some packages)

Forms:

> `RUN <command>` (execute via `shell`)

__or__:

> `RUN ["executable", "param1", "param2"]` (`exec` form)

Which form to use?
- `shell` - if we want to run `shell` (usually `bash`) command like `apt-get install`
- `exec` - if the base image has no shell __or__ we don't want string munging


In [ ]:
# Equivalent
RUN /bin/bash -c 'source $HOME/.bashrc; echo $HOME'
RUN ["/bin/bash", "-c", "echo hello"]

### [ENTRYPOINT](https://docs.docker.com/engine/reference/builder/#entrypoint)


> __Defines entrypoint (command which will be run) WHEN CONTAINER IS CREATED FROM AN IMAGE__

Forms:

> `ENTRYPOINT ["executable", "param1", "param2"]` (preferred `exec` form)

__or__

> `ENTRYPOINT command param1 param2` (`shell` form)

- Container runs as an executable (which you should always aim to do, more on that later!)
- You should always specify it (unless you want to use `shell`)
- __Either of `ENTRYPOINT` or `CMD` is needed__

#### `exec` form

- __Does not invoke shell__, hence it is not dependent on it
- __Allows us to use optional `CMD`__ (in a second, after command)

In [ ]:
FROM ubuntu
# When we run a container from the image, top -b will be run
ENTRYPOINT ["top", "-b"]

### [CMD](https://docs.docker.com/engine/reference/builder/#cmd)


> __Specifies default arguments to entrypoint (if any) WHICH USER CAN OVERRIDE DURING `docker run`__

Forms:

> `CMD ["executable","param1","param2"]` (specify `executable` as `entrypoint`, whole command can be overridden)

__or__

> `CMD ["param1","param2"]` (as default parameters to ENTRYPOINT, only those could be overridden)

__or__ 

> `CMD command param1 param2` (shell form, __discouraged__ as users is unable to override)

In [ ]:
FROM ubuntu
ENTRYPOINT ["top", "-b"]
CMD ["-c"]

Now if we `run` the container from the above image, command `top -b -c` will be run.

- `top -b` __will always run__
- `-c` can be changed to some other flag/command via `docker run`

Let's see how `CMD` interacts with `ENTRYPOINT` for a better understanding:

Note: `/bin/sh -c` is just command which executes the proceeding code in the terminal

![](images/docker_entrypoint_cmd_interaction.png)

### [COPY](https://docs.docker.com/engine/reference/builder/#copy)


> __Allows users to specify which file(s) or directories should be copied into the image from host system__

> `COPY <src> <destination>`

Often idiom `COPY . .` is used, which copies file from context location to current working directory inside container. 

It might look like we're copying something to the same location, but that's not what's happening. We essentially have two file structures which we are moving files between. The file system which the first argument to `COPY` refers to is the build context (wherever you run `docker build` from). The file system which the second argument to `COPY refers to is the file system within your docker container.

### Other commands

There are a few others, notably:
- `LABEL <key>=<value>` - allows us to add metadata to our image (like author, maintainer, way of contacting)
- `WORKDIR dir` - sets working directory to a different one
- `ENV <key>=<value>` - environment variable readable throughout the concrete build stage
- `EXPOSE <port>` - `EXPOSE 80` would expose port `80` inside the container for others to connect (usually a person running `docker` command will specify which ports to expose and connect, hence this one isn't used very often).

## Commands tips



### Know how to use cache

> __Some commands invalidate cache__ and when this happens, every step following it will have to be re-run when you create the image

Let's look at the example Dockerfile below (similar to the first one):

In [ ]:
FROM ubuntu:18.04

RUN apt-get update
COPY . .

RUN apt-get install -y --no-install-recommends python3
RUN rm -rf /var/lib/apt/lists/*

Now, no matter what happens, `python3` will be installed during each `docker build`, because:

> Docker has no mechanism to check whether `context` for `COPY` command changed

Instead we could do what we've seen at the very beginning __as Python installation is not dependent on the context__.

> __If possible, put `COPY` statements AFTER setting up OS dependencies__

### Chain commands together

> Whenever possible __chain multiple command using `&&`__ so they are all in a single `RUN` directive

Docker works in a similar fashion to `git`, __it only stashes changes (additions) to the system__.

This is often undesirable, because:
- Temporary files are left out and increase image's size (`rm -rf /var/lib/apt/lists/*` seen at the beginning of the lesson)
- Containers __are less of a black box__, which means attackers can analyze Docker system easier and find it's weak points

__Main command to look out for is `RUN`, most of the commands (like `LABEL`) DO NOT create an additional layer__

## Small, self-contained images


> __The smaller the image, the better__

Imagine we have to run `10` containers from a single image published image. Now, if the image weighs `1GB` we would need to use at least `10GB` of bandwidth.

Compare that to an image of `10MB`. Other pros include:
- Smaller latency for users (setup takes considerably shorter)
- Easier to recreate a fleet of containers (more on that during Kubernetes)
- Easier to replace a failed container

There is one killer feature of Docker which helps us achieve it, namely...


## Multi-stage builds



> First image builds the application (creating an artifact), while the second copies it and sets it up for running in a container

The easiest approach is to look at an example `Dockerfile`:

In [ ]:
# FIRST (BUILDER) STAGE
FROM golang:1.7.3 AS builder

# Obtain golang code
WORKDIR /go/src/github.com/alexellis/href-counter/
RUN go get -d -v golang.org/x/net/html  
COPY app.go .
# Compile is as a single exectuable file called app
RUN CGO_ENABLED=0 GOOS=linux go build -a -installsuffix cgo -o app .

# SECOND STAGE
# alpine is a very slim file system (few MB) great for lightweight deployment
FROM alpine:latest  

# Setup only bare necessities
RUN apk --no-cache add ca-certificates
WORKDIR /root/
# Copy self-contained app into the smaller image
COPY --from=builder /go/src/github.com/alexellis/href-counter/app .
# Setup the application as container's ENTRYPOINT
ENTRYPOINT ["./app"]  

> __You should use multi-stage builds wherever possible!__

### Pros

- Drastically reduces image size
- Simplifies maintenance

### Cons

- __Mainly usable for compilable language__ (sorry Python :( ) like Go, C++
- __Even better with statically linked__ (e.g. everything is contained in a single executable)
- __Hard to make it work with ML/DL__ as those require a lot of dependencies

One way to go around it is to use `torchscript` and `C++` PyTorch's frontend (or neural network conversion to Tensorflow) and changing the language

> Be aware, that this approach is currently really hard and might bring you a lot of headaches!

> __Remember deployment is not only about neural networks, there are other things (like servers, databases etc.) that might benefit from this approach!__

> __Online registry with many official and third party images uploaded and ready to be run as containers (or act as base)__

![](images/dockerhub_main_page.png)

One can `docker run` those images directly and those will be downloaded automatically:

In [ ]:
docker run busybox:latest ls -la

### Tips

- Use official images if possible
- Use smallest image fitting the job (e.g. `alpine` instead of `ubuntu` if possible)
- __Check unofficial images__ (or roll out your own)

## Docker Commands


Now, that we know a little bit about images, we may dive into Docker's command line interface.

> __In new Docker version (`>=1.13`) the command line was redesigned in order to be more readable and grouped logicially__

__High level most important commands__:
- [`docker image SUBCOMMAND`](https://docs.docker.com/engine/reference/commandline/image/) - manage `Docker` images (like building or inspecting them)
- [`docker container SUBCOMMAND`](https://docs.docker.com/engine/reference/commandline/container/) - manage `Docker` containers (creating from image, stopping, restarting, killing, inspecting etc.)
- [`docker volume SUBCOMMAND`](https://docs.docker.com/engine/reference/commandline/volume/) - manages volumes (persistent data storage which might be shared and attached to Docker containers)
- [`docker network SUBCOMMAND`](https://docs.docker.com/engine/reference/commandline/network/) - manage `Docker` networks (like creating, inspecting, listing etc., __not covered here, we will use Kubernetes for network related tasks__)

__High level less important commands__:
- [`docker config SUBCOMMAND`](https://docs.docker.com/engine/reference/commandline/config/) - configuration of Docker
- [`docker stack`](https://docs.docker.com/engine/reference/commandline/stack/) - manage multiple containers as whole (__not covered, we will use Kubernetes instead__)
- [`docker secret`](https://docs.docker.com/engine/reference/commandline/secret/) - manage secrets (like passwords and other sensitive data inside containers/images)
- [`docker system`](https://docs.docker.com/engine/reference/commandline/system/) - manage Docker itself (how much space is used, cleaning images/containers etc.)

Those `SUBCOMMAND`s are also available inside `docker` (e.g. `docker image build` is equivalent to `docker build`) and may be a source of confusion

> __Check out documentation for `docker build` instead of `docker image build` as it has more information, BUT USE THE LATTER AS IT IS WAY MORE READABLE!__


### docker image build


> `docker image build [OPTIONS] PATH | URL | -`

As seen previously one can use it to `build` `IMAGE` from `Dockerfile` __and__ `context`.

Except building from local, one can also build from:
- `github`: `docker build github.com/creack/docker-firefox`
- `tar.gaz`: `docker build -f ctx/Dockerfile http://server/ctx.tar.gz` (here context is on a different server)
- From `stdin` (no context in this case): `docker build - < Dockerfile`

#### Options

- `-t` - tag the image (__always use it!__): `docker build -t whenry/fedora-jboss:latest -t whenry/fedora-jboss:v2.1 .` (multiple tags supported)
- `-f` - specify different file: `docker build -f dockerfiles/Dockerfile.debug -t myapp_debug .`; __useful for separate production, testing, debugging images!__
- `--build-arg` - pass arguments to the build stage (`ARG` above): `docker build --build-arg HTTP_PROXY=http://10.20.30.2:1234 .`




### docker container run



__The most important command you will all the time with A LOT of options!__

> __REMINDER:__ "Docker runs processes in isolated containers. A container is a process which runs on a host." 

> `docker container run [OPTIONS] IMAGE[:TAG] [COMMAND] [ARG...]`

Using `OPTIONS` developer can override defaults set by the image creator, including, but not limited to:
- detached or foreground running
- network settings
- runtime constraints
- command run

> `COMMAND` specifies a command to be passed to image entrypoint

> `ARG` arguments passed to the command

We have seen examples above

#### Running container interactively

> `docker container run` can attach streams (`STDIN`, `STDOUT`, `STDERR`) of container and attach a terminal so we can interact with the container

- `-a NAME_OF_STREAM`
- `-t` allocate pseudo TTY (terminal)
- `-i` keep STDIN open even if not attached to CLI

`/bin/bash` specifies the entrypoint of TTY we have attached (or attached to)

### Options

- `--name NAME` - specify name for the container (__always do this__)
- `--rm` - remove container after exit (__usually do this__); otherwise it will prevail in your operating system not doing anything
- `-d` - run in a detached mode (default runs in foreground, equivalent to `-d=false`)
- `-m` - set memory limit (`docker run -it -m 300M ubuntu:14.04 /bin/bash`), there are also flags for other resources
- `-e NAME=VALUE` - pass environment variable to the container (`export today=Wednesday; docker run -e "deep=purple" -e today --rm alpine env` would allow us to use `$today` inside the container)
- `--entrypoint` - override default image entrypoint (`docker run -it --entrypoint /bin/bash example/redis`); __reset via `--entrypoint=""`!__

> For more check out [`docker run` reference](https://docs.docker.com/engine/reference/run/)

### Exit codes

> If your `docker run` fails check the return code to know where to look for bugs

- `125` - error within the daemon (e.g. wrong flag passed; `docker run --foo busybox`)
- `126` - contained comment cannot be __invoked__ (`docker run busybox /etc` - it is a directory, not a command)
- `127` - contained comment cannot be found (`docker run busybox foo` - no command `foo`)

> Otherwise return code of contained comment will be returned (usually `0` if executed correctly)

## Docker volumes

We have been through a lot of content, there is one thing left... __Docker may create artifacts__ (like metrics from training or data after preprocessing).

__How to get them out of container?__

There are two options:
- using `docker container cp` command
- using volumes

> __Volumes are persistent storage shared between host machine and Docker container(s)__

Benefits should be obvious:
- data sharing between containers and hosts (example: data preprocessing container creates dataset, neural network container trains our model on it)
- we can copy to/from the containers and do the live updates of their data content
- we can set the volume to be readable only for increased security

### docker volume create

> Create volume __which includes contents of the directory it was created in__

(see `exercise` to check how to verify what's inside the volume)

In [ ]:
docker volume create docker_lesson

Now, we can __mount__ the volume to `/lesson` directory inside container (and list it's contents):

In [ ]:
docker container run --rm -v docker_lesson:/lesson busybox ls /lesson

### Mounting

Let's take a look at a couple ways to `mount` the volume using `--mount`:

In [ ]:
docker run \
  --name devtest \
  --mount source=myvol2,target=/app \
  nginx:latest

## Exercise

- Read the following aliases with comments and copy them to your aliases location
- Run `--help` with them to know a little bit more (and check docs/google if something interests you)

In [ ]:
# Images
alias di="docker image" # General for docker images

alias dib="docker image build" # Build docker image
alias dil="docker image ls" # List docker images (check --help)
alias dip="docker image push" # Push NAME image
alias dirm="docker image rm" # Remove NAME image
alias dirmall="docker image prune -a" # Remove all images not used by containers

# Containers
alias dc="docker container" # General for docker containers

alias dcr="docker container run" # Run container from an IMAGE image
alias dccp="docker container cp" # Copy data from src to dst inside container
alias dce="docker container exec" # Execute COMMAND inside container
alias dci="docker container inspect" # Inspect container
alias dck="docker container kill"  # Kill container
alias dcl="docker container ls" # List all available containers
alias dcs="docker container stop" # Stop running container

alias dcrma='docker ps -a -q | xargs sudo docker rm' # Remove all non-running containers

# Volumes
alias dv="docker volume" # General volume command

alias dvc="docker volume create" # Create NAMED volume
alias dvl="docker volume ls" # List all available volumes
alias dvrm="docker volume rm" # Remove volumes

# List files inside the created volume
dvi(){
  docker run --rm -i -v="$1":/tmp/myvolume busybox find /tmp/myvolume
}

# System
alias dsi="docker system info" # Display system-wide information
alias dsdf="docker system df" # How much images & containers take in terms of space
alias dsp="docker system prune" # Remove every unused image/containerj